### Linear Model Selection Example 2.2
We begin with the *full model}, that is $ \mathcal{M}_{10} $, which contains all $ p $ predictors of the \data{Credit} data set
\begin{align*}
balance
&=\beta_{0}+\beta_{1}\cdot income +\beta_{2}\cdot limit +\beta_{3}\cdot rating +\beta_{4}\cdot cards\\
&\quad+\beta_{5}\cdot age +\beta_{6}\cdot education +\beta_{7}\cdot gender +\beta_{8}\cdot student \\
&\quad +\beta_{9}\cdot married +\beta_{10}\cdot ethnicity +\epsilon
\end{align*}
Then we remove one predictor variable from the model.

We create a new function, similar to the **add\_one** function, which removes each predictor separately from the full model. All created functions will be saved in **LMS\_def**, and imported when needed. 

In [1]:
import pandas as pd
import numpy as np
from LMS_def import *

# Load data
df = pd.read_csv('./data/Credit.csv')

# Convert Categorical variables
df = pd.get_dummies(data=df, drop_first=True, 
                    prefix=('Gender_', 'Student_', 
                            'Married_', 'Ethnicity_'))

x_full = df.drop(columns='Balance')
y = df['Balance']

def drop_one(x, y, scoreby='RSS'):
    ''' Remove possible predictors from x, 
    Fit a linear model on y using fit_linear_reg
    Returns Dataframe showing scores as well as predictor 
    to drop in order to keep the best model '''
    # Predefine DataFrame
    x_labels = x.columns
    zeros = np.zeros(len(x_labels))
    results = pd.DataFrame(
        data={'Predictor': x_labels.values, 'BIC': zeros, 
               'AIC': zeros, 'R2': zeros, 
               'R2_adj': zeros, 'RSS': zeros})

    # For every predictor find RSS and R^2
    for i in range(len(x_labels)):
        x_i = x.drop(columns=x_labels[i])
        results.iloc[i, 1:] = fit_linear_reg(x_i, y)
    
    # Depending on where we scoreby, we select the highest or lowest
    if scoreby in ['RSS', 'AIC', 'BIC']:
        worst = x_labels[results[scoreby].argmin()]
    elif scoreby in ['R2', 'R2_adj']:
        worst = x_labels[results[scoreby].argmax()]
    
    return results, worst 

results, worst1 = drop_one(x_full, y)

print(results[['Predictor', 'AIC', 'R2', 'RSS']], 
      '\n\nWorst predictor is:',  worst1)

               Predictor          AIC        R2           RSS
0             Unnamed: 0  4821.370391  0.955102  3.786730e+06
1                 Income  5361.643903  0.826689  1.461702e+07
2                  Limit  4853.911857  0.951296  4.107672e+06
3                 Rating  4826.005381  0.954578  3.830864e+06
4                  Cards  4837.510427  0.953253  3.942650e+06
5                    Age  4825.143746  0.954676  3.822621e+06
6              Education  4820.935924  0.955150  3.782619e+06
7         Gender__Female  4821.391825  0.955099  3.786933e+06
8           Student__Yes  5214.259751  0.880104  1.011204e+07
9           Married__Yes  4821.188761  0.955122  3.785011e+06
10      Ethnicity__Asian  4821.918353  0.955040  3.791921e+06
11  Ethnicity__Caucasian  4821.043224  0.955138  3.783634e+06 

Worst predictor is: Education


Now we remove the  *least useful* variable which is the one that yields the reduced regression model with the lowest RSS or the highest $ R^2 $. This predictor represents the most *redundant* variable because its removal improves the model most significantly with respect to the RSS. In this case, this is the predictor **education**. 

Thus, we obtain the model $ \mathcal{M}_{9} $ which is given by
\begin{align*}
balance
&=\beta_{0}+\beta_{1}\cdot income +\beta_{2}\cdot limit +\beta_{3}\cdot rating +\beta_{4}\cdot cards \\
&\quad+\beta_{5}\cdot age +\beta_{6}\cdot gender +\beta_{7}\cdot student +\beta_{8}\cdot married \\
&\quad +\beta_{9}\cdot ethnicity +\epsilon
\end{align*}

We now remove another variable from this model. To do so we first need to update the reference model which means dropping the selected predictor from the full set. Subsequently, we run the same procedure again.

In [2]:
# Remove the chosen predictor from the list of options
x_red1 = x_full.drop(columns=worst1, errors='ignore')

results, worst2 = drop_one(x_red1, y)

print(results[['Predictor', 'AIC', 'R2', 'RSS']], 
      '\n\nWorst predictor is:',  worst2)

               Predictor          AIC        R2           RSS
0             Unnamed: 0  4819.857603  0.955047  3.791345e+06
1                 Income  5359.644014  0.826689  1.461703e+07
2                  Limit  4851.966157  0.951290  4.108230e+06
3                 Rating  4824.775477  0.954491  3.838246e+06
4                  Cards  4835.976056  0.953198  3.947242e+06
5                    Age  4823.685440  0.954615  3.827801e+06
6         Gender__Female  4819.862964  0.955046  3.791396e+06
7           Student__Yes  5213.014039  0.879877  1.013112e+07
8           Married__Yes  4819.758863  0.955058  3.790410e+06
9       Ethnicity__Asian  4820.411139  0.954985  3.796596e+06
10  Ethnicity__Caucasian  4819.562597  0.955080  3.788550e+06 

Worst predictor is: Ethnicity__Caucasian


#### Optional:
We could again repeat this procedure until one 1 predictor is left.

In [3]:
# Remove the chosen predictor from the list of options
x_red2 = x_red1.drop(columns=worst2, errors='ignore')

results, worst3 = drop_one(x_red2, y)

print(results, '\n\nWorst predictor is:',  worst3)

          Predictor          BIC          AIC        R2    R2_adj  \
0        Unnamed: 0  4858.512384  4818.597738  0.954964  0.953924   
1            Income  5398.135774  5358.221128  0.826439  0.822434   
2             Limit  4890.496911  4850.582266  0.951215  0.950089   
3            Rating  4863.290746  4823.376101  0.954422  0.953371   
4             Cards  4874.514290  4834.599645  0.953125  0.952044   
5               Age  4862.350937  4822.436291  0.954529  0.953480   
6    Gender__Female  4858.381505  4818.466860  0.954978  0.953939   
7      Student__Yes  5251.005635  5211.090990  0.879854  0.877082   
8      Married__Yes  4858.201340  4818.286694  0.954999  0.953960   
9  Ethnicity__Asian  4858.336298  4818.421652  0.954983  0.953945   

            RSS  
0  3.798367e+06  
1  1.463813e+07  
2  4.114563e+06  
3  3.844014e+06  
4  3.953400e+06  
5  3.834993e+06  
6  3.797125e+06  
7  1.013307e+07  
8  3.795415e+06  
9  3.796695e+06   

Worst predictor is: Married__Yes


Analogue to Example 2.1, We can automatically drop the n-worst predictors using SequentialFeatureSelector from **sklearn.feature_selection**, setting *direction='backward'*. 
The chosen predictors are returned in the support_ attribute.
**Note**: If *None* features are selected, the algorithm automatically choses half number of features given. 

We choose the model that has the smallest RSS. This turns out to be the 
case when we remove the predictor variable **Ethnicity\_Caucasian** from the reference model. We then obtain the model $ \mathcal{M}_8 $:
\begin{align*}
balance
&=\beta_{0}+\beta_{1}\cdot income +\beta_{2}\cdot limit +\beta_{3}\cdot rating +\beta_{4}\cdot cards \\
&\quad+\beta_{5}\cdot age +\beta_{6}\cdot gender +\beta_{7}\cdot student \\
&\quad +
\beta_{8}\cdot Married
+\epsilon
\end{align*}

We iterate this procedure until *no* predictor variable is left in the regression model. This iterative procedure yields 11 different models $\mathcal{M}_{0},\mathcal{M}_{1},\ldots,\mathcal{M}_{10}$. We identify the *best* among these models on the basis of the AIC, which we will discuss later. 
 
The selection procedure can also be performed using **sklearn.feature\_selector**

In [4]:
from sklearn.feature_selection import SequentialFeatureSelector 
from sklearn.linear_model import LinearRegression

# define Linear Regression Model in sklearn
linearmodel = LinearRegression()
# Sequential Feature Selection using sklearn
sfs = SequentialFeatureSelector(linearmodel, n_features_to_select=3, 
                          direction='backward')
sfs.fit(x_full, y)

# Print Chosen variables
print(x_full.columns[sfs.support_].values)


['Income' 'Limit' 'Student__Yes']


The regression model with three predictors contains the variables **income**, **limit** and **student**. This model differs from the model we obtained by forward stepwise selection. In this model the variable **rating** appears instead of the variable **limit**. 